In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cars = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cardekho_2-22344.csv')




In [4]:
cars

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage(km/ltr/kg),engine,max_power,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248.0,74,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497.0,78,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.00,1396.0,90,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8123,Hyundai i20 Magna,2013,320000,110000,Petrol,Individual,Manual,First Owner,18.50,1197.0,82.85,5.0
8124,Hyundai Verna CRDi SX,2007,135000,119000,Diesel,Individual,Manual,Fourth & Above Owner,16.80,1493.0,110,5.0
8125,Maruti Swift Dzire ZDi,2009,382000,120000,Diesel,Individual,Manual,First Owner,19.30,1248.0,73.9,5.0
8126,Tata Indigo CR4,2013,290000,25000,Diesel,Individual,Manual,First Owner,23.57,1396.0,70,5.0


In [5]:
cars.columns

Index(['name', 'year', 'selling_price', 'km_driven', 'fuel', 'seller_type',
       'transmission', 'owner', 'mileage(km/ltr/kg)', 'engine', 'max_power',
       'seats'],
      dtype='object')

In [6]:
cars.isnull().sum()

,0
name,0
year,0
selling_price,0
km_driven,0
fuel,0
seller_type,0
transmission,0
owner,0
mileage(km/ltr/kg),221
engine,221


In [7]:
y= cars.selling_price
X= cars.drop('selling_price', axis=1)

In [8]:
from sklearn.model_selection import train_test_split
X_train_f, X_test_f, y_train, y_test= train_test_split(X,y, test_size=0.2, train_size=0.8, random_state=2)

categorical_col= [col for col in X_train_f.columns if X_train_f[col].nunique()<10 and X_train_f[col].dtype== 'object']

numarical_col= [col for col in X_train_f.columns if X_train_f[col].dtype in ['int64', 'float64']]

#keep the selected
my_col= categorical_col + numarical_col
X_train= X_train_f[my_col].copy()
X_test= X_test_f[my_col].copy()


In [9]:
#for missing values(numarical) => imputing with most freaquant
from sklearn.impute import SimpleImputer
#for catigoricalfeatures 1- imputeing 2-encoding using OHE so we need to use piplining
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

myImputer= SimpleImputer(strategy='most_frequent')

catigorical_tran= Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('one hot', OneHotEncoder(handle_unknown='ignore'))])

#To Apply these on the whole columns => I will use columnTransformer
preprocessor = ColumnTransformer(transformers= [('num', myImputer, numarical_col),('cat', catigorical_tran, categorical_col)])


To build the **Model**

In [10]:
from sklearn.ensemble import RandomForestRegressor

model=RandomForestRegressor(random_state=0)

In [11]:
from sklearn.metrics import mean_absolute_error
my_pipline= Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])


In [12]:
def get_mae(max_leaf_nodes, X_train, X_test, y_train, y_test):
    model = RandomForestRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    my_pipline.fit(X_train, y_train)
    pred=my_pipline.predict(X_test)
    score=mean_absolute_error(y_test, pred)
    return (score)


In [13]:
cand_max_leaf=[1,2,3,4,5,10]
best_leaf=0
min_mae=50000
for max_leaf_nodes in (1,2,3,4,5,10):
  my_mae= get_mae(max_leaf_nodes, X_train, X_test, y_train, y_test)
  if my_mae<min_mae:
    best_leaf=max_leaf_nodes
    min_mae=my_mae
print(best_leaf, min_mae)


0 50000
